# TensorFlow基础使用

* 加载TensorFlow

In [1]:
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

1.10.0


1. 创建Tensor: Tensor又称张量，简单可以理解为多维矩阵(数组),TensorFlow中图计算数据流动的基本单元就是Tensor.

使用`tf.constant()`可以快速定义Tensor,这里定义两个二维矩阵`a`和`b`,使用`tf.matmul()`进行矩阵乘法操作,分别计算`ab`和`ba`的结果.可以发现打印出来的四个结果都是`Tensor`类型, 在TensorFlow中Tensor是不能修改的, 这一点和Variable不同.

In [2]:
a = tf.constant([[1.0, 2.0, 3.0]])
b = tf.constant([[1.0], [2.0], [3.0]])
c = tf.matmul(a, b)
d = tf.matmul(b, a)
print(a)
print(b)
print(c)
print(d)

tf.Tensor([[1. 2. 3.]], shape=(1, 3), dtype=float32)
tf.Tensor(
[[1.]
 [2.]
 [3.]], shape=(3, 1), dtype=float32)
tf.Tensor([[14.]], shape=(1, 1), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [2. 4. 6.]
 [3. 6. 9.]], shape=(3, 3), dtype=float32)


2. 创建Variable:Variable是TensorFlow保存模型状态和参数的基本单元,可以简单地理解为TensorFlow中所有可训练的参数都使用Variable保存.

使用`tf.get_variable()`创建Variable, 类似于Tensor示例进行两个矩阵乘法操作的展示, 
输出结果为Tensor类型, 其他和Tensor操作十分类似. Tensor和Variable可以混合使用.

In [3]:
va = tf.get_variable('var_a', [1, 3], dtype=tf.float32,
                    initializer=tf.constant_initializer([[1.0, 2.0, 3.0]]))
vb = tf.get_variable('var_a', [3, 1], dtype=tf.float32,
                    initializer=tf.constant_initializer([[1.0], [2.0], [3.0]]))
vc = tf.matmul(va, vb)
vd = tf.matmul(vb, va)
print(va)
print(vb)
print(vc)
print(vd)

print(tf.matmul(a, vb))
print(tf.matmul(b, va))

<tf.Variable 'var_a:0' shape=(1, 3) dtype=float32, numpy=array([[1., 2., 3.]], dtype=float32)>
<tf.Variable 'var_a:0' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [2.],
       [3.]], dtype=float32)>
tf.Tensor([[14.]], shape=(1, 1), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [2. 4. 6.]
 [3. 6. 9.]], shape=(3, 3), dtype=float32)
tf.Tensor([[14.]], shape=(1, 1), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [2. 4. 6.]
 [3. 6. 9.]], shape=(3, 3), dtype=float32)


3. 利用Tensor和Variable进行计算:使用Tensor创建输入,Variable作为可训练参数,简单模拟函数$f(x)=Ax+b$的操作.

In [4]:
x = tf.constant([[1.0], [2.0], [3.0]])
A = tf.get_variable('A', [4, 3], dtype=tf.float32,
                initializer=tf.random_uniform_initializer(-1.0, 1.0))
b = tf.get_variable('b', [4, 1], dtype=tf.float32,
                initializer=tf.random_uniform_initializer(-1.0, 1.0))
f = tf.matmul(A, x) + b
print(x)
print(A)
print(b)
print(f)

tf.Tensor(
[[1.]
 [2.]
 [3.]], shape=(3, 1), dtype=float32)
<tf.Variable 'A:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.71516156, -0.9896023 , -0.40463972],
       [ 0.09952044,  0.0820322 , -0.29023957],
       [-0.6299825 , -0.5753865 ,  0.3008921 ],
       [-0.07673454,  0.17851329,  0.18940139]], dtype=float32)>
<tf.Variable 'b:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.89026093],
       [-0.15398908],
       [-0.2103827 ],
       [-0.2951069 ]], dtype=float32)>
tf.Tensor(
[[-1.5877013 ]
 [-0.76112294]
 [-1.0884619 ]
 [ 0.5533893 ]], shape=(4, 1), dtype=float32)


4. Session(会话):Session(会话)是进行图计算的环境(上下文),有些类似运行程序的沙盒的作用,
    其提供图计算需要的资源并保留相关状态,进行模型存储和加载的时候,也是以Session作为保存和加载对象. 这里需要注意的是,之前运行的程序都是在使用`tf.enable_eager_execution()`启动交互模式运行的结果, 现在使用的是使用普通Session运行的结果. 在普通Session模式下, 先构建计算图, 然后初始化必须的参数, 最后使用`Session.run()`方法计算指定计算图中的节点得到结果.

In [5]:
with tf.Session() as session:
    x = tf.constant([[1.0], [2.0], [3.0]])
    A = tf.get_variable('A', [4, 3], dtype=tf.float32,
                    initializer=tf.random_uniform_initializer(-1.0, 1.0))
    b = tf.get_variable('b', [4, 1], dtype=tf.float32,
                    initializer=tf.random_uniform_initializer(-1.0, 1.0))
    f = tf.matmul(A, x) + b
    print(x)
    print(A)
    print(b)
    print(f)
    
    session.run(tf.global_variables_initializer())
    answer, x_val, A_val, b_val = session.run([f, x, A, b])
    print(x_val)
    print(A_val)
    print(b_val)
    print(answer)

Tensor("Const:0", shape=(3, 1), dtype=float32)
<tf.Variable 'A:0' shape=(4, 3) dtype=float32>
<tf.Variable 'b:0' shape=(4, 1) dtype=float32>
Tensor("add:0", shape=(4, 1), dtype=float32)
[[1.]
 [2.]
 [3.]]
[[-0.44130492  0.12559676  0.7421856 ]
 [ 0.6288309  -0.8435364  -0.8658266 ]
 [ 0.07993364 -0.52946496  0.89745855]
 [ 0.33952975 -0.60447335 -0.69467545]]
[[ 0.9536092 ]
 [ 0.39151216]
 [ 0.07432961]
 [-0.9359064 ]]
[[ 2.9900546]
 [-3.2642095]
 [ 1.787709 ]
 [-3.8893497]]
